## Preferencias Británicas
Se consideran los siguientes vectores de atributos binarios:
*(scones, cerveza, whisky, avena, futbol)*

In [24]:
import pandas as pd

df = pd.read_excel('../../res/tp1/PreferenciasBritanicos.xlsx', sheet_name='Hoja1')

nationality_names = {'E': 'Escocesa', 'I': 'Inglesa'}
row_count = df.shape[0]
# Get number of records from each nationality
nationality_count = df['Nacionalidad'].value_counts()
nationalities = nationality_count.keys()
# Get relative frequency for each nationality (ie: P(v))
relative_freq = nationality_count.apply(lambda x: (x + 1) / (row_count + len(nationalities)))
# Get conditional probabilities for each attribute, given a nationality (ie: P(a_i | v))
cond_probs = df.groupby('Nacionalidad').apply(lambda x: (x.sum() + 1) / (len(x) + 2))

def classify(prefs):
    '''Returns the most likely nationality
    
    Parameters
    ----------
    prefs : A tuple with preferences (ie: (1,1,1,0,1)) and dimensions 1x5
    '''
    preferences = pd.DataFrame(data=(prefs,), columns=('scones', 'cerveza', 'wiskey', 'avena', 'futbol'))

    vs = nationality_count.astype(dtype=float, copy=True)  # We only want a dataframe with this dimensions, this is just the easiest way
    for nationality in nationalities:
        # P(a1,a2,a3,a4,a5 | vj) * P(Vj)
        vs[nationality] = preferences.apply(
            lambda x: get_cond_prob_by_preference(x, nationality)
        ).prod() * relative_freq[nationality]
    
    vnb = vs.idxmax(axis=1)
    
    conj_prob_preference = vs.sum() # (ie: P(a1 = 1, a2 = 1, a3 = 1, a4 = 0, a5 = 1))
    for nationality in nationalities:
        # P(Vj | a1,a2,a3,a4,a5)
        print(f"La probabilidad de tener nacionalidad {nationality_names[nationality]} es {(vs[nationality] / conj_prob_preference * 100):.5} %")
    
    return vnb

def get_cond_prob_by_preference(preference, nationality):
    '''Returns P(Ai|Vj) if preference is 1, P(~Ai|Vj) otherwise
    
    Parameters
    ----------
    preference : value with column name and just one binary (0,1) value in the subindex [0]
    nationality: either 'E' or 'I'
    '''
    cond_prob = cond_probs.loc[nationality, preference.name]  # P(Ai | Vj)
    return cond_prob if preference.values[0] == 1 else (1 - cond_prob)


Se clasificó el ejemplo x1 = (1, 0, 1, 1, 0) determinando si correspondía a las preferencias de una persona inglesa o escosesa:

In [25]:
print(f"La persona debe ser {nationality_names[classify((1,0,1,1,0))]}")

La probabilidad de tener nacionalidad Escocesa es 76.025 %
La probabilidad de tener nacionalidad Inglesa es 23.975 %
La persona debe ser Escocesa


Se clasificó el ejemplo x2 = (0, 1, 1, 0, 1) determinando si correspondía a las preferencias de una persona inglesa o escosesa:

In [26]:
print(f"La persona debe ser {nationality_names[classify((0,1,1,0,1))]}")

La probabilidad de tener nacionalidad Escocesa es 16.541 %
La probabilidad de tener nacionalidad Inglesa es 83.459 %
La persona debe ser Inglesa
